## Import required packages

In [1]:
import openai
import requests
from bs4 import BeautifulSoup
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
from pymilvus import MilvusClient, DataType
from tqdm import tqdm

## Set up vector DB (`Milvus`)

### Start `Milvus` service

In order to start `Milvus` service run the following commands:

```
mkdir milvus && cd milvus
curl -sfL https://raw.githubusercontent.com/milvus-io/milvus/master/scripts/standalone_embed.sh -o standalone_embed.sh
bash standalone_embed.sh start
```

### Connect to the `Milvus`

In [2]:
client = MilvusClient(
    uri='http://127.0.0.1:19530',
    token='root:Milvus'
)

### Init a shema for images storage

In [3]:
ARTICLE_TEXT_MAX_LENGTH = 1500

In [4]:
article_schema = MilvusClient.create_schema()

article_schema.add_field(field_name='id', datatype=DataType.INT64, is_primary=True, auto_id=True)
article_schema.add_field(field_name='vector', datatype=DataType.FLOAT_VECTOR, dim=1536)
article_schema.add_field(field_name='article_text', datatype=DataType.VARCHAR, max_length=ARTICLE_TEXT_MAX_LENGTH*2)
article_schema.add_field(field_name='article_url', datatype=DataType.VARCHAR, max_length=200)

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1536}}, {'name': 'article_text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 3000}}, {'name': 'article_url', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 200}}], 'enable_dynamic_field': False}

### Enable indexing on `id` and `vector` fields 

In [5]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name='vector',
    index_type='FLAT',
    metric_type='COSINE'
)

### Create collection (if not created yet)

In [6]:
COLLECTION_NAME = 'articles_bihus'

In [7]:
client.drop_collection(
    collection_name=COLLECTION_NAME,
)

In [8]:
if not client.has_collection(collection_name=COLLECTION_NAME):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        schema=article_schema,
        index_params=index_params
    )

### Verify that the collection is ready

In [9]:
client.get_load_state(
    collection_name=COLLECTION_NAME
)

{'state': <LoadState: Loaded>}

## Set up text embedding extraction model and LLM agent

We will use `OpenAI` API for both text embedding extraction (`text-embedding-3-small`) and LLM (`gpt-4o-mini`)

In [10]:
OPENAI_API_KEY = '<OPENAI_API_KEY>'

### LLM agent (i.e. Chatbot)

In [11]:
chat = ChatOpenAI(
    openai_api_key = OPENAI_API_KEY,
    model = 'gpt-4o-mini'
)

/tmp/ipykernel_4128805/4116842513.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


#### Testing

In [12]:
messages = [
    SystemMessage(content = 'You are a helpful assistant.'),
    HumanMessage(content = 'Hi AI, how are you?'),
    AIMessage(content = 'I am great thank you. How can I help you?'),
    
    HumanMessage(content = 'I would like to understand the structure of Ukrainian government')
]

In [13]:
agent_answer = chat(messages)

print(agent_answer.content)

/tmp/ipykernel_4128805/1253727881.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_answer = chat(messages)


The structure of the Ukrainian government is defined by its Constitution, which was adopted in 1996. Ukraine is a unitary parliamentary republic, which means that it has a system of government in which the executive branch derives its democratic legitimacy from the legislature (Verkhovna Rada) and is held accountable to it.

Here’s an overview of the key components of the Ukrainian government structure:

### 1. **Executive Branch**
- **President**: The President of Ukraine is the head of state and is elected by popular vote for a five-year term (with the possibility of re-election). The President has significant powers, including the ability to appoint the Prime Minister (with the approval of the Verkhovna Rada), oversee foreign policy, and serve as the commander-in-chief of the armed forces.
  
- **Cabinet of Ministers**: The Cabinet, headed by the Prime Minister, is responsible for the day-to-day administration of the government. The Prime Minister is appointed by the President with 

### Text embedding extraction model

In [14]:
embed_model = OpenAIEmbeddings(
    model='text-embedding-3-small',
    openai_api_key=OPENAI_API_KEY
)

/tmp/ipykernel_4128805/3757787941.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embed_model = OpenAIEmbeddings(


#### Testing

In [15]:
test_texts = [
    'Lorem ipsum dolor sit amet, consectetur adipiscing elit.',
    'Vestibulum vulputate ullamcorper tortor, sit amet vehicula velit dignissim ut.',
    'Suspendisse potenti.'
]

test_embedings = embed_model.embed_documents(test_texts)

len(test_embedings), len(test_embedings[0])

(3, 1536)

## Parse "Bihus.Info" articles

In [16]:
BIHUS_URL = 'https://bihus.info/news/'
HTTP_HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [17]:
def get_articles_urls():
    page = requests.get(BIHUS_URL, headers=HTTP_HEADERS)
    
    soup = BeautifulSoup(page.content, 'html.parser')
    a_tags = soup.find_all('a', class_='bi-latest-post__link')
    
    return {a['href'] for a in a_tags}

In [18]:
def get_article_text(article_url: str):
    page = requests.get(article_url, headers=HTTP_HEADERS)
    soup = BeautifulSoup(page.content, 'html.parser')
    article_paragraphs = soup.find(class_='bi-single-content').find_all('p')
    
    paragraphs = [paragraph.text for paragraph in article_paragraphs]

    return '\n'.join(paragraphs)

In [19]:
articles_urls = get_articles_urls()

len(articles_urls), list(articles_urls)[:5]

(26,
 ['https://bihus.info/rozsliduvannya-bihus-info-pro-stezhennya-z-boku-sbu-vyznaly-krashhym-na-naczionalnomu-konkursi-zhurnalistskyh-rozsliduvan/',
  'https://bihus.info/genprokuror-zareyestruvav-dva-kryminalnyh-provadzhennya-shhodo-narodnogo-deputata-romana-ivanisova-pislya-syuzhetu-bihus-info/',
  'https://bihus.info/pislya-syuzhetu-bihus-info-u-minoborony-povidomyly-pro-zvilnennya-lyudyny-nardepa-isayenka/',
  'https://bihus.info/pidnyattya-podatkiv-zamist-borotby-z-tinnyu-yak-za-roky-getmanczeva-u-radi-najsirishi-zony-ekonomiky-shhe-bilshe-potemnily/',
  'https://bihus.info/bihus-info-pokazalo-provalnu-robotu-i-nezadeklarovani-statky-spivrobitnykiv-servisnogo-czentru-mvs/'])

In [20]:
article_text_dict = {
    get_article_text(articles_url): articles_url
    for articles_url in tqdm(articles_urls)
}

len(article_text_dict), sum([len(text) for text in article_text_dict.keys()])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:03<00:00,  8.23it/s]


(26, 53790)

## Vectorize articles

### Split articles into chunks

In [21]:
def split_string(text: str, max_size: int) -> list[str]:
    parts = []

    sentence_temp = ''
    for sentence in text.split('.'):
        if len(sentence_temp) + len(sentence) > max_size:
            parts.append(sentence_temp.strip())
            sentence_temp = sentence
        else:
            sentence_temp = f'{sentence_temp} {sentence}'
    
    if sentence_temp:
        parts.append(sentence_temp.strip())
    
    return parts

In [22]:
article_chunks_dict = {}

for article_text, article_url in article_text_dict.items():
    article_chunks = split_string(article_text, ARTICLE_TEXT_MAX_LENGTH)
    for article_chunk in article_chunks:
        article_chunks_dict[article_chunk] = article_url

len(article_chunks_dict), sum(len(chunk) for chunk in article_chunks_dict.keys())

(49, 53363)

### Extract embeddings from text

In [23]:
text_embeddings = embed_model.embed_documents(article_chunks_dict.keys())

len(text_embeddings), len(text_embeddings[0])

(49, 1536)

### Load data into `Milvus`

In [24]:
data = []

for paragraph_embedding, (article_chunk, article_url) in zip(text_embeddings, article_chunks_dict.items()):
    data += [{
        'vector': paragraph_embedding,
        'article_text': article_chunk,
        'article_url': article_url
    }]

len(data)

49

In [25]:
_ = client.insert(collection_name=COLLECTION_NAME, data=data)

### Demonstration of the similiarity search

In [26]:
def get_similiar(query: str, k: int = 3) -> list[tuple[int, int, float]]:
    query_vector = embed_model.embed_documents([query])[0]

    candidates = client.search(
        collection_name=COLLECTION_NAME,
        data=[query_vector],
        limit=k,
        output_fields=['article_text', 'article_url']
    )[0]

    return [(candidate['entity']['article_text'], candidate['entity']['article_url']) 
        for candidate in candidates
    ]

In [27]:
query = 'З якими проблемами стикався Київський метрополітен?'

get_similiar(query, k=3)

[('Нагадаємо, минулого компанія ТОВ “КБ “Теплоенергоавтоматика” – потрапила у скандал через підряди від КП «Київський метрополітен», зокрема через засекречений контракт на 1,5 млрд грн на ремонт «червоної гілки» метро  Журналістам Bihus Info вдалося з’ясувати не тільки багатомільйонні завищення цін на кабельну продукцію, але й зв’язок фірми безпосередньо із керівником Метрополітену Віктором Брагінським  Виявилося, що компанія оформлена на Тетяну Белецьку, балерину й педагога Національної опери, а за сумісництвом – тещу давнього бізнес-партнера Брагінського, Кирила Кривця \nЦікава в рамках уже нового “газового” контракту деталь: до того, як очолити Метрополітен, Брагінський працював у структурах НАК “Нафтогаз”, там же у той період працював і Кривець  Оператор ГТС, від якого ТОВ “КБ “ТЕА” має отримати 1,6 мільярда – створений із філії “Укртрансгазу”, тобто історично також належить до структури групи “Нафтогазу” \nПісля оприлюднення журналістського розслідування – кримінальні провадження 

# RAG!!!

In [28]:
def get_answer(query: str, k: int = 3):
    results = get_similiar(query, k=k)

    article_texts = [article_text for article_text, _ in results]
    sources = {article_url for _, article_url in results}
    
    messages = [
        SystemMessage(content = 'Ти журналіст-розслідувач, який дає відповіді на запитання щодо різних розслідувань'),
        HumanMessage(content = 'Привіт, друже. Як справи?'),
        AIMessage(content = 'Чудово, готовий до нових питань. Що саме тебе цікавить?'),
        # prompt
        HumanMessage(content=augment_prompt(article_texts))
    ]
    
    agent_answer = chat(messages)
    
    return {
        'question': query,
        'answer': agent_answer.content, 
        'source': sources
    }
    
    
def augment_prompt(text: str):
    source_knowledge = '\n'.join(text)
    
    return f'''
        <system_prompt>Використовуючи наданий контекст, дай відповідь на поставлене питання<system_prompt>
        
        <context>{source_knowledge}<context>
        
        <question>{query}<question>
    '''

In [29]:
# query = 'Хто купував нерухомість в ЖК "Комфорт Таун"?'
query = 'Чому затримали Новохацького?'

get_answer(query)

{'question': 'Чому затримали Новохацького?',
 'answer': 'Новохацького затримали через його роль посередника у передачі відкату за виконання робіт, фінансованих з бюджету Херсона. Він запропонував директору однієї з місцевих компаній "допомогти" отримати підряд від КП "Херсонтеплоенерго" в обмін на 22,5% від суми договору. Згідно з інформацією, він також погрожував, що підприємець не зможе безперешкодно вести бізнес в Херсоні та області у разі відмови. Затримання відбулося під час передачі відкату, що стало підставою для дій Служби безпеки України.',
 'source': {'https://bihus.info/nabu-i-sap-rozsliduyut-jmovirne-nezakonne-zbagachennya-prokurora-guczulyaka/',
  'https://bihus.info/rozsliduvannya-bihus-info-pro-stezhennya-z-boku-sbu-vyznaly-krashhym-na-naczionalnomu-konkursi-zhurnalistskyh-rozsliduvan/',
  'https://bihus.info/sbu-zatrymala-figuranta-syuzhetu-bihus-info-pro-rozpyly-na-budivnycztvi-shkil-hersonshhyny-za-zbir-vidkativ/'}}